<a href="https://colab.research.google.com/github/tomasonjo/blogs/blob/master/pyg2neo/Movie_recommendations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* Updated to GDS 2.0 version
* Link to original blog post: https://towardsdatascience.com/integrate-neo4j-with-pytorch-geometric-to-create-recommendations-21b0b7bc9aa

In [ ]:
!pip install sentence_transformers neo4j
!pip install torch-scatter torch-sparse torch-cluster torch-geometric -f https://data.pyg.org/whl/torch-1.10.0+cpu.html

Looking in links: https://data.pyg.org/whl/torch-1.10.0+cpu.html


In [ ]:
import torch
import pandas as pd
import numpy as np
from torch.nn import Linear
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer

import torch_geometric.transforms as T
from torch_geometric.nn import SAGEConv, to_hetero

from torch_geometric.data import HeteroData
from torch_geometric.transforms import ToUndirected, RandomLinkSplit

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


# Integrate Neo4j with PyTorch Geometric to create recommendations
## Leverage the power of PyTorch Geometric to develop and train custom Graph Neural Networks for your application
I have wanted to write about the PyTorch Geometric (pyG) ever since I saw they announced their collaboration with Stanford University on their workshop. The PyTorch Geometric (pyG) is a library built upon PyTorch to help you easily write and train custom Graph Neural Networks for your applications. In this blog post, I will present how you can fetch data from Neo4j to create movie recommendations in PyTorch Geometric.
The graph we will be working with is the MovieLens dataset, which is handily available as a Neo4j Sandbox project. Not knowing before, there is an example in pyG that also uses the MovieLens dataset for a link prediction task. The links between users and movies have a rating score. We can then use the graph neural network model to predict which unseen movies a user is likely to rate high and then use that information to recommend them.
The main goal of this post is to show you how to convert a Neo4j graph into a heterogeneous pyG graph. As a side goal, we will also prepare some of the node features in Neo4j using the Graph Data Science library and export them to pyG.
## Agenda
* Develop movie embeddings to capture movie similarity based on the actors and directors
* Export Neo4j Graph and construct a heterogeneous pyG graph
* Train the GNN model in pyG
* Create predictions and optionally store them back to Neo4j

As always, I have prepared a Google Colab notebook if you want to follow along with the examples in this post.
## Develop movie embeddings to capture movie similarity based on the actors and directors
First, you need to create and open the [Recommendations project in Neo4j Sandbox](https://sandbox.neo4j.com/?usecase=recommendations) that has the MovieLens dataset already populated. Next, you need to define the Neo4j connection in the notebook.

In [ ]:
from neo4j import GraphDatabase

url= 'bolt://3.86.43.255:7687'
user = 'neo4j'
password = 'company-science-journals'

driver = GraphDatabase.driver(url, auth=(user, password))

def fetch_data(query, params={}):
  with driver.session() as session:
    result = session.run(query, params)
    return pd.DataFrame([r.values() for r in result], columns=result.keys())


The link prediction example in pyG uses word embeddings of the title and one-hot encoding of genres as the node features. To make it a bit more interesting, we will also develop movie node features that encapsulate the similarity of actors and directors. We could, similarly to genres, one-hot encode actors and directors. But instead, we will choose another route to capture movie similarities based on the actors that appeared in the movies. In this example, we will use the FastRP embedding models to produce node embeddings. If you want to learn more about the FastRP algorithm, I suggest you check out [this excellent article](https://towardsdatascience.com/behind-the-scenes-on-the-fast-random-projection-algorithm-for-generating-graph-embeddings-efb1db0895) by my friend CJ Sullivan.
The FastRP algorithm will only consider the bipartite network of movies and persons and ignore genres and ratings. This way, we make sure to capture the movie similarity based on only the actors and directors that appeared in the movie. First, we need to project the GDS in-memory graph.

In [ ]:
fetch_data("""
CALL gds.graph.project('movies', ['Movie', 'Person'], 
  {ACTED_IN: {orientation:'UNDIRECTED'}, DIRECTED: {orientation:'UNDIRECTED'}})
""")

,nodeProjection,relationshipProjection,graphName,nodeCount,relationshipCount,createMillis
0,"{'Movie': {'properties': {}, 'label': 'Movie'}...","{'DIRECTED': {'orientation': 'UNDIRECTED', 'ag...",movies,28172,91834,172


Now we can go ahead and execute the FastRP algorithm on the projected graph and store the results back in the database.

In [ ]:
fetch_data("""
CALL gds.fastRP.write('movies', {writeProperty:'fastrp', embeddingDimension:56})
""")

,nodeCount,nodePropertiesWritten,createMillis,computeMillis,writeMillis,configuration
0,28172,28172,0,394,1322,"{'writeConcurrency': 4, 'normalizationStrength..."


After executing the FastRP algorithm, each movie node has a node property fastrp that contains the embeddings that encapsulate the similarity based on the present actors and directors.
## Export Neo4j Graph and construct a heterogeneous pyG graph
I've taken a lot of inspiration for constructing a custom heterogenous pyG graph from this example. The example creates a pyG graph from multiple CSV files. I have simply rewritten the example to fetch the data from Neo4j instead of CSV files.
The generic function to create node mapping and features while retrieving data from Neo4j is the following:

In [ ]:
def load_node(cypher, index_col, encoders=None, **kwargs):
    # Execute the cypher query and retrieve data from Neo4j
    df = fetch_data(cypher)
    df.set_index(index_col, inplace=True)
    # Define node mapping
    mapping = {index: i for i, index in enumerate(df.index.unique())}
    # Define node features
    x = None
    if encoders is not None:
        xs = [encoder(df[col]) for col, encoder in encoders.items()]
        x = torch.cat(xs, dim=-1)

    return x, mapping

And similarly, the function to create edge index and features is:

In [ ]:
def load_edge(cypher, src_index_col, src_mapping, dst_index_col, dst_mapping,
                  encoders=None, **kwargs):
    # Execute the cypher query and retrieve data from Neo4j
    df = fetch_data(cypher)
    # Define edge index
    src = [src_mapping[index] for index in df[src_index_col]]
    dst = [dst_mapping[index] for index in df[dst_index_col]]
    edge_index = torch.tensor([src, dst])
    # Define edge features
    edge_attr = None
    if encoders is not None:
        edge_attrs = [encoder(df[col]) for col, encoder in encoders.items()]
        edge_attr = torch.cat(edge_attrs, dim=-1)

    return edge_index, edge_attr

As mentioned, the code is almost identical to the pyG example. I've just changed that the Pandas DataFrame is constructed from data retrieved from Neo4j instead of CSV files. In the next step, we have to define the feature encoders.

In [ ]:
class SequenceEncoder(object):
    # The 'SequenceEncoder' encodes raw column strings into embeddings.
    def __init__(self, model_name='all-MiniLM-L6-v2', device=None):
        self.device = device
        self.model = SentenceTransformer(model_name, device=device)

    @torch.no_grad()
    def __call__(self, df):
        x = self.model.encode(df.values, show_progress_bar=True,
                              convert_to_tensor=True, device=self.device)
        return x.cpu()

In [ ]:
class GenresEncoder(object):
    # The 'GenreEncoder' splits the raw column strings by 'sep' and converts
    # individual elements to categorical labels.
    def __init__(self, sep='|'):
        self.sep = sep

    def __call__(self, df):
        genres = set(g for col in df.values for g in col.split(self.sep))
        mapping = {genre: i for i, genre in enumerate(genres)}

        x = torch.zeros(len(df), len(mapping))
        for i, col in enumerate(df.values):
            for genre in col.split(self.sep):
                x[i, mapping[genre]] = 1
        return x

In [ ]:
class IdentityEncoder(object):
    # The 'IdentityEncoder' takes the raw column values and converts them to
    # PyTorch tensors.
    def __init__(self, dtype=None, is_list=False):
        self.dtype = dtype
        self.is_list = is_list

    def __call__(self, df):
        if self.is_list:
            return torch.stack([torch.tensor(el) for el in df.values])
        return torch.from_numpy(df.values).to(self.dtype)

Finally, we can fetch the data from Neo4j and construct user mappings and features that will be used as input to the pyG heterogeneous graph. We will begin by constructing the user nodes input. They have no node features available, so we don't have to include any encoders.

In [ ]:
user_query = """
MATCH (u:User) RETURN u.userId AS userId
"""

user_x, user_mapping = load_node(user_query, index_col='userId')

Next, we will construct the movie's mapping and features.

In [ ]:
movie_query = """
MATCH (m:Movie)-[:IN_GENRE]->(genre:Genre)
WITH m, collect(genre.name) AS genres_list
RETURN m.movieId AS movieId, m.title AS title, apoc.text.join(genres_list, '|') AS genres, m.fastrp AS fastrp
"""

movie_x, movie_mapping = load_node(
    movie_query, 
    index_col='movieId', encoders={
        'title': SequenceEncoder(),
        'genres': GenresEncoder(),
        'fastrp': IdentityEncoder(is_list=True)
    })

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/39.3k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/286 [00:00<?, ?it/s]

With the movies, we have several node features. First, the Sequence encoder uses the sentence-transformers library to produce word embeddings based on the titles. The genres are one-hot-encoded using the Genre encoder, and lastly, we simply transform the FastRP embeddings into the correct structure to be able to use them in PyTorch Geometric.

Before we can construct the pyG graph, we have to fetch the information about the ratings, which are represented as weighted links between users and movies.

In [ ]:
rating_query = """
MATCH (u:User)-[r:RATED]->(m:Movie) 
RETURN u.userId AS userId, m.movieId AS movieId, r.rating AS rating
"""

edge_index, edge_label = load_edge(
    rating_query,
    src_index_col='userId',
    src_mapping=user_mapping,
    dst_index_col='movieId',
    dst_mapping=movie_mapping,
    encoders={'rating': IdentityEncoder(dtype=torch.long)},
)

We have all the information we require. So now, we can go ahead and build a heterogeneous pyG graph. In a heterogeneous graph, distinct types of nodes contain different features.

In [ ]:
data = HeteroData()
# Add user node features for message passing:
data['user'].x = torch.eye(len(user_mapping), device=device)
# Add movie node features
data['movie'].x = movie_x
# Add ratings between users and movies
data['user', 'rates', 'movie'].edge_index = edge_index
data['user', 'rates', 'movie'].edge_label = edge_label
data.to(device, non_blocking=True)

HeteroData(
  user={ x=[671, 671] },
  movie={ x=[9125, 460] },
  (user, rates, movie)={
    edge_index=[2, 100004],
    edge_label=[100004]
  }
)

The process of constructing a heterogeneous pyG graph seems easy enough. First, we define node features of each node type and then add any relationships between those nodes. Remember, the GNNs require all of the nodes to contain node features. So here is one example of what you could do for user nodes with no pre-existing features.

As with all the Machine Learning flows, we have to perform the train/test data split. The pyG library makes this very easy with the RandomLinkSplit method.

In [ ]:
data = ToUndirected()(data)
del data['movie', 'rev_rates', 'user'].edge_label  # Remove "reverse" label.

# 2. Perform a link-level split into training, validation, and test edges.
transform = RandomLinkSplit(
    num_val=0.1,
    num_test=0.1,
    neg_sampling_ratio=0.0,
    edge_types=[('user', 'rates', 'movie')],
    rev_edge_types=[('movie', 'rev_rates', 'user')],
)
train_data, val_data, test_data = transform(data)

The pyG graph is prepared. Now, we can go ahead and define our GNN. I have simply copied the definition from the pyG example. The GNN will predict ratings between 0 and 5 of movies by users. We can think of that as a link prediction task, where we are predicting the relationship property of new links between users and movies. Once we have defined the GNN, we can go ahead and train our model.

In [ ]:
class GNNEncoder(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv((-1, -1), hidden_channels)
        self.conv2 = SAGEConv((-1, -1), out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x


class EdgeDecoder(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.lin1 = Linear(2 * hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, 1)

    def forward(self, z_dict, edge_label_index):
        row, col = edge_label_index
        z = torch.cat([z_dict['user'][row], z_dict['movie'][col]], dim=-1)

        z = self.lin1(z).relu()
        z = self.lin2(z)
        return z.view(-1)

class Model(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.encoder = GNNEncoder(hidden_channels, hidden_channels)
        self.encoder = to_hetero(self.encoder, data.metadata(), aggr='sum')
        self.decoder = EdgeDecoder(hidden_channels)

    def forward(self, x_dict, edge_index_dict, edge_label_index):
        z_dict = self.encoder(x_dict, edge_index_dict)
        return self.decoder(z_dict, edge_label_index)

In [ ]:
weight = torch.bincount(train_data['user','rates', 'movie'].edge_label)
weight = weight.max() / weight

def weighted_mse_loss(pred, target, weight=None):
    weight = 1. if weight is None else weight[target].to(pred.dtype)
    return (weight * (pred - target.to(pred.dtype)).pow(2)).mean()

In [ ]:
model = Model(hidden_channels=64).to(device)

In [ ]:
# Due to lazy initialization, we need to run one model step so the number
# of parameters can be inferred:
with torch.no_grad():
    model.encoder(train_data.x_dict, train_data.edge_index_dict)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
def train():
    model.train()
    optimizer.zero_grad()
    pred = model(train_data.x_dict, train_data.edge_index_dict,
                 train_data['user', 'rates', 'movie'].edge_label_index)
    target = train_data['user', 'rates', 'movie'].edge_label
    loss = weighted_mse_loss(pred, target, weight)
    loss.backward()
    optimizer.step()
    return float(loss)

In [ ]:
@torch.no_grad()
def test(data):
    model.eval()
    pred = model(data.x_dict, data.edge_index_dict,
                 data['user', 'rates', 'movie'].edge_label_index)
    pred = pred.clamp(min=0, max=5)
    target = data['user', 'rates', 'movie'].edge_label.float()
    rmse = F.mse_loss(pred, target).sqrt()
    return float(rmse)

Since the dataset is not that big, it shouldn't take long to train the model. If you can, use the GPU mode in the Google Colab environment.

In [ ]:
for epoch in range(1, 300):
    loss = train()
    train_rmse = test(train_data)
    val_rmse = test(val_data)
    test_rmse = test(test_data)
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Train: {train_rmse:.4f}, '
          f'Val: {val_rmse:.4f}, Test: {test_rmse:.4f}')

Epoch: 001, Loss: 19.7843, Train: 3.2615, Val: 3.2684, Test: 3.2474
Epoch: 002, Loss: 16.5262, Train: 2.3640, Val: 2.3769, Test: 2.3551
Epoch: 003, Loss: 9.3904, Train: 1.1806, Val: 1.1700, Test: 1.1679
Epoch: 004, Loss: 10.0532, Train: 1.1701, Val: 1.1806, Test: 1.1639
Epoch: 005, Loss: 6.7872, Train: 1.7661, Val: 1.7819, Test: 1.7603
Epoch: 006, Loss: 6.7277, Train: 2.0491, Val: 2.0635, Test: 2.0421
Epoch: 007, Loss: 7.7062, Train: 1.9760, Val: 1.9906, Test: 1.9696
Epoch: 008, Loss: 7.3644, Train: 1.6654, Val: 1.6815, Test: 1.6611
Epoch: 009, Loss: 6.3172, Train: 1.2720, Val: 1.2882, Test: 1.2700
Epoch: 010, Loss: 5.8945, Train: 1.0770, Val: 1.0887, Test: 1.0755
Epoch: 011, Loss: 6.5527, Train: 1.0791, Val: 1.0920, Test: 1.0785
Epoch: 012, Loss: 6.3446, Train: 1.2318, Val: 1.2482, Test: 1.2313
Epoch: 013, Loss: 5.6389, Train: 1.4683, Val: 1.4845, Test: 1.4665
Epoch: 014, Loss: 5.6257, Train: 1.6203, Val: 1.6355, Test: 1.6176
Epoch: 015, Loss: 5.8645, Train: 1.6264, Val: 1.6413, Test:

Lastly, we will predict new links between users and movies and store results back to Neo4j. We will only consider links where the predicted rating is equal to 5.0, which is the highest possible rating, for movie recommendations.

In [ ]:
num_movies = len(movie_mapping)
num_users = len(user_mapping)

reverse_movie_mapping = dict(zip(movie_mapping.values(),movie_mapping.keys()))
reverse_user_mapping = dict(zip(user_mapping.values(),user_mapping.keys()))

results = []

for user_id in range(0,num_users): 

    row = torch.tensor([user_id] * num_movies)
    col = torch.arange(num_movies)
    edge_label_index = torch.stack([row, col], dim=0)

    pred = model(data.x_dict, data.edge_index_dict,
                 edge_label_index)
    pred = pred.clamp(min=0, max=5)

    user_neo4j_id = reverse_user_mapping[user_id]

    mask = (pred == 5).nonzero(as_tuple=True)

    ten_predictions = [reverse_movie_mapping[el] for el in  mask[0].tolist()[:10]]
    results.append({'user': user_neo4j_id, 'movies': ten_predictions})
    

I've only selected the first ten recommendations for each user to make it simple and not have to import tens of thousands of relationships back to Neo4j. One important thing to note is that we don't filter out existing links or ratings in our predictions, so we'll just skip them during import to the Neo4j database. Let's import those predictions to Neo4j under the RECOMMEND relationships.

In [ ]:
import_predictions_query = """
UNWIND $data AS row
MATCH (u:User {userId: row.user})
WITH u, row
UNWIND row.movies AS movieId
MATCH (m:Movie {movieId: movieId})
WITH u,m
// filter out existing links
WHERE NOT (u)-[:RATED]->(m)
MERGE (u)-[:RECOMMEND]->(m)
"""

fetch_data(import_predictions_query, {'data': results})

""


## Conclusion
PyTorch Geometric is a powerful library that allows you to develop and train custom Graph Neural Networks applications. I am looking forward to exploring it more and seeing all the applications I can create using it.
The MovieLens dataset contains several node features we haven't used, like the release date or the movie budget that you could test out. You could also try and change the GNN definition. Let me know if you find any exciting approaches that work for you.